In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader,Subset
import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats("svg")

/tmp/ipykernel_14129/615205814.py:10: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  display.set_matplotlib_formats("svg")


In [2]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [ ]:
transform = T.Compose([
    T.ToTensor(),  # normalizes to range [0,1]
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Import the data and simultaneously apply the transform
trainset = torchvision.datasets.STL10(
    root='./data', download=True, split='train', transform=transform
)

testset = torchvision.datasets.STL10(
    root='./data', download=True, split='test', transform=transform
)

# Transform the datasets into dataloaders
batchsize = 32
train_loader = DataLoader(trainset, batch_size=batchsize, shuffle=True, drop_last=True)
test_loader = DataLoader(testset, batch_size=256)

In [ ]:
X,y=next(iter(train_loader))

In [ ]:
fig,axs=plt.subplots(4,4,figsize=(10,10))
for i, ax in enumerate(axs.flatten()):
    # Extract the image (need to transpose it back to 32x32x3)
    pic = X.data[i].numpy().transpose((1, 2, 0))
    
    # Undo normalization
    pic = pic - np.min(pic)  
    pic = pic / np.max(pic)
    
    # Extract the label
    label = trainset.classes[y[i]]
    
    # Show the image
    ax.imshow(pic)
    
    # Add the label as text on the image
    ax.text(0, 0, label, ha='left', va='top', fontweight='bold', color='k', backgroundcolor='y')
    
    # Turn off axis
    ax.axis('off')

# Adjust the layout to prevent overlap
plt.tight_layout()

# Display the figure
plt.show()



In [ ]:
resnet=torchvision.models.resnet18(pretrained=True)

In [ ]:
for p in resnet.paramateres():
  p.requires_grad=False

In [ ]:
resnet.fc=nn.Linear(512,10)

In [ ]:
resnet.to(device)

In [ ]:
lossfun=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(resnet.parameters(),lr=0.001,momentum=.9)

In [ ]:
numepochs=10
trainLoss=torch.zeros(numepochs)
testLoss=torch.zeros(numepochs)
testAcc=torch.zeros(numepochs)
trainAcc=torch.zeros(numepochs)
for epochi in range(numepochs):
  resnet.train()
  batchLoss=[]
  batchAcc=[]
  for X ,y in train_loader:
    X=X.to(device)
    y=y.to(device)
    yHat=resnet(X)
    loss=lossfun(yHat,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    batchLoss.append(loss.item())
    batchAcc.append(torch.mean((torch.argmax(yHat,axis=1) == y).float()).item())
  trainLoss[epochi] = np.mean(batchLoss)
  trainAcc[epochi] =100*np.mean(batchAcc)
  resnet.eval()
  batchAcc=[]
  batchLoss=[]
  for X,y in test_loader:
    X=X.to(device)
    y=y.to(device)
    with torch.no_grad():
      yHat= resnet(X)
      loss=lossfun(yHat,y)
    batchLoss.append(loss.item())
    batchAcc.append(torch.mean((torch.argmax(yHat,axis=1) == y).float()).item())
  testLoss[epochi] = np.mean(batchLoss)
  testAcc[epochi] =100*np.mean(batchAcc)
  print(f'Finshed Epoch {epochi+1}/{numepochs}. Test Accuracy {testAcc[epochi]:.2f}%')





In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 5))

# Plotting Train and Test Loss
ax[0].plot(trainLoss, 's-', label='Train')
ax[0].plot(testLoss, 'o-', label='Test')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss (MSE)')
ax[0].set_title('Model loss')
ax[0].legend()

# Plotting Train and Test Accuracy
ax[1].plot(trainAcc, 's-', label='Train')
ax[1].plot(testAcc, 'o-', label='Test')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy (%)')
ax[1].set_title(f'Final model train/test accuracy: {trainAcc[-1]:.2f}/{testAcc[-1]:.2f}%')
ax[1].legend()

# Add a main title for the whole figure
plt.suptitle('Pretrained ResNet-18 on STL10 data', fontweight='bold', fontsize=14)

# Show the plots
plt.show()
